In [ ]:
!pip install git+https://github.com/Refefer/fastxml.git

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd

train_data = pd.read_csv('/kaggle/input/authattrdata/train_tweets.txt', sep = '\t', header = None)

In [ ]:
y = train_data[0]
tweets = train_data[1]

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

print('labels:', len(set(y)))
print('samples:', len(tweets))

distt = Counter(y)
counts = []

for key, val in distt.most_common():
    counts.append(val)

plt.plot(counts)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased", do_lower_case = False)

n_tweets = []
for tweet in tweets:
    nTweet = tokenizer.encode(tweet)
    n_tweets.append(' '.join([str(x) for x in nTweet]))

In [ ]:
import json

X = []
for labl, tweet in zip(y, n_tweets):
    x = {"title": tweet,
       "tags": [str(labl)]}
    X.append(json.dumps(x))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(X, test_size = 0.25, random_state = 42)

In [ ]:
with open('/kaggle/working/fxml_train.json', 'w') as fp:
    for line in X_train:
        fp.write(line + '\n')

with open('/kaggle/working/fxml_test.json', 'w') as fp:
    for line in X_test:
        fp.write(line + '\n')

In [ ]:
!python3 "/kaggle/input/fxmlscript/fxml.py" "/kaggle/working/auth.model" "/kaggle/working/fxml_train.json" train --iters 5 --trees 10 --label-weight propensity --alpha 1e-4 --threads 4 --leaf-classifiers

In [ ]:
!python3 "/kaggle/input/fxmlscript/fxml.py" "/kaggle/working/auth.model" "/kaggle/working/fxml_test.json" inference